In [27]:
# Kinetic Component Analysis in Julia
#### Own Julia code translated from the Python code found in Kinetic Component Analysis, Marcos López de Prado & Riccardo Rebonato - Journal of Investing (2016). and only changed slightly to allow for user input with regarding to the size of A (i.e. the number of degrees of the Taylor expansion).

In [65]:
#### Load packages
using IJulia
using StateSpaceModels
using Random
using Distributions
using Plots
using LinearAlgebra

In [98]:
# y = Z * a + e ~ N(0, H)
# a = T * a + R * n ~ N(0, Q)

fitKCA = function(t, z, q, a = 2)
    # 1) Set up matrices A, H and Q
    h = (t[end] - t[1])/length(z)
    H = zeros(1, a + 1) ; H[1, 1] = 1
    A = zeros(a + 1, a + 1)
    for i in 1:a+1
        A[1:a-i+2, i:a+1] += Matrix(I*1.0/factorial(i-1)*h^(i-1), a - i + 2, a - i + 2)
    end
    Q = Matrix(I*q, a + 1, a + 1)

    # 2-4) Apply filter, EM estimates and smooth
    kf = statespace(StateSpaceModel(z, H, A, Q))
    x_mean  = kf.smoother.alpha
    x_covar = kf.smoother.V

    # 6) Std series
    x_std = zeros(length(z), a + 1)
    for i in 1:a+1
        x_std[:, i] = sqrt.(kf.smoother.V[i, i, :])
    end

    return (x_mean, x_std, x_covar)
end

#69 (generic function with 2 methods)

In [80]:
getPeriodic = function(periods, nobs, scale, seed = 0)
    t = range(0, stop = pi*periods, length = nobs)
    s = sin.(t)
    z = s .+ rand(Normal(0.0, scale^2), nobs)
    return (t, s, z)
end

#59 (generic function with 2 methods)

In [99]:
nobs = 300
periods = 10

t, s, z = getPeriodic(periods, nobs, 0.5)

kf = fitKCA(t, z, 0.001, 2)

                  StateSpaceModels.jl v0.3.2                  
 (c) Raphael Saavedra, Guilherme Bodin, and Mario Souto, 2019 
--------------------------------------------------------------
            Starting state-space model estimation.            
    Initiating maximum likelihood estimation with 3 seeds.    
--------------------------------------------------------------
||    seed    |     log-likelihood      |      time (s)     ||
||       1    |           -93.9743      |          0.20     ||
||       2    |           -93.9635      |          0.34     ||
||       3    |           -95.5237      |          0.61     ||
--------------------------------------------------------------
           Maximum likelihood estimation complete.            
                  Log-likelihood:   -93.9635                  
                             AIC:   195.9270                  
                             BIC:   210.7421                  
             End of state-space model estimation.      

([-0.04652260492957516 1.2769328577438843 -0.7084160361580869; 0.08331315000090334 1.2027699986700213 -0.7087030157404115; … ; 0.04630770504544765 1.6307340707883318 1.3974139732322568; 0.2247416285066045 1.7770742849688321 1.397415603605959], [0.14915606485864236 0.41942119524552884 0.7667278592740674; 0.11924530843939676 0.3552265617801981 0.7158609848873156; … ; 0.11924712694941383 0.35563998143100606 0.7667330480244001; 0.1491568416001696 0.4198127165349982 0.814885169613568], [0.02224753168411553 -0.04897108248616282 0.05363432888973213; -0.04897108250158701 0.17591413902118802 -0.2668981433111047; 0.053634328889749 -0.2669001937131722 0.5878716101869941]

[0.014219443584806868 -0.02899550728034228 0.02902935958991293; -0.028996350229135714 0.1261859101941809 -0.20566845589200966; 0.02902893263672013 -0.20567103238136042 0.5124569496838376]

[0.009628982284283438 -0.0158454874031122 0.010746787709649652; -0.015845486964735755 0.08855904742813436 -0.15330832681502216; 0.01074681174

3×3×300 Array{Float64,3}:
[:, :, 1] =
  0.0247205   -0.0529434    0.00171385
 -0.0529434    0.225472    -0.00832396
  0.00171385  -0.00832985   0.0199199

[:, :, 2] =
  0.0161142    -0.0311726    0.000951173
 -0.0311704     0.170378    -0.00640724
  0.000950674  -0.00641377   0.0199199

[:, :, 3] =
  0.0114588    -0.0173707    0.000388804
 -0.0173706     0.129503    -0.00475661
  0.000388784  -0.00475715   0.0199187

...

[:, :, 298] =
 0.0114588    0.0133196   0.000388759
 0.0133196    0.16927     0.00432569
 0.000388759  0.00432569  0.0199164

[:, :, 299] =
 0.0161146    0.0292767   0.000950948
 0.0292767    0.22396     0.00624232
 0.000950948  0.00624232  0.0199164

[:, :, 300] =
 0.0247205   0.0529434   0.00171385
 0.0529434   0.289042    0.00832796
 0.00171385  0.00832796  0.0199164